In [1]:
import subprocess
import time
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.webdriver.support.ui import Select
import re

In [2]:
# Define paths
chrome_path = "/Applications/Google Chrome.app/Contents/MacOS/Google Chrome"
user_data_dir = "/Users/federicopessina/ChromeProfile"  # Use a dedicated folder
remote_debugging_port = "9222"
factiva_url = "https://librarysearch.lse.ac.uk/discovery/fulldisplay?vid=44LSE_INST:44LSE_VU1&tab=Everything&docid=alma99129371110302021&context=L&search_scope=MyInstitution&lang=en"

# Launch Chrome with remote debugging
subprocess.Popen([
    chrome_path,
    f"--remote-debugging-port={remote_debugging_port}",
    f"--user-data-dir={user_data_dir}",
    factiva_url
])

# Give time for Chrome to open and for you to log in
print("Chrome launched. Log in manually, then press Enter to continue...")
input()

Chrome launched. Log in manually, then press Enter to continue...


[6755:4062982:0527/223334.166271:ERROR:net/cert/internal/trust_store_mac.cc:817] Error parsing certificate:
ERROR: Failed parsing extensions


DevTools listening on ws://127.0.0.1:9222/devtools/browser/1dc4aea1-6467-49dd-bb46-c3dd58ea52e9
[6755:4062959:0527/223337.381320:ERROR:chrome/browser/policy/cloud/fm_registration_token_uploader.cc:179] Client is missing for kUser scope
I0000 00:00:1748381617.430047 4062985 voice_transcription.cc:58] Registering VoiceTranscriptionCapability
Trying to load the allocator multiple times. This is *not* supported.
[6755:4062986:0527/223337.477643:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT
Created TensorFlow Lite XNNPACK delegate for CPU.
Attempting to use a delegate that only supports static-sized tensors with a graph that has dynamic-sized tensors (tensor#-1 is a dynamic-sized tensor).


''

In [3]:
options = Options()
options.add_experimental_option("debuggerAddress", "localhost:9222")

# Adjust path to your chromedriver
service = Service("/opt/homebrew/bin/chromedriver")
driver = webdriver.Chrome(service=service, options=options)

time.sleep(3)

In [4]:
print(driver.current_url)
print(len(driver.find_elements(By.ID, "dr")))  # dropdown
print(len(driver.find_elements(By.ID, "frd"))) # from-day field
print(len(driver.find_elements(By.ID, "laTab"))) # language tab
driver.switch_to.window(driver.window_handles[-1])  # last tab
print(driver.current_url)

https://global-factiva-com.lse.idm.oclc.org/sb/default.aspx?lnep=hp
1
1
1
https://global-factiva-com.lse.idm.oclc.org/sb/default.aspx?lnep=hp


In [5]:
#### Change the date range ####
data_range = {'Start': '2020-01-01', 'End': '2020-03-01'}
start_day, start_month, start_year = data_range['Start'].split('-')[2], data_range['Start'].split('-')[1], data_range['Start'].split('-')[0]
end_day, end_month, end_year = data_range['End'].split('-')[2], data_range['End'].split('-')[1], data_range['End'].split('-')[0]

time.sleep(2)

# Select "Custom" date range
select = Select(driver.find_element(By.ID, "dr"))
select.select_by_visible_text("Enter date range...")
time.sleep(1.5)

# Fill in "From" date fields
driver.find_element(By.ID, 'frd').clear()
driver.find_element(By.ID, 'frd').send_keys(start_day)

driver.find_element(By.ID, 'frm').clear()
driver.find_element(By.ID, 'frm').send_keys(start_month)

driver.find_element(By.ID, 'fry').clear()
driver.find_element(By.ID, 'fry').send_keys(start_year)

# Fill in "To" date fields
driver.find_element(By.ID, 'tod').clear()
driver.find_element(By.ID, 'tod').send_keys(end_day)

driver.find_element(By.ID, 'tom').clear()
driver.find_element(By.ID, 'tom').send_keys(end_month)

driver.find_element(By.ID, 'toy').clear()
driver.find_element(By.ID, 'toy').send_keys(end_year)



In [6]:
#### Change language ####
time.sleep(2)
driver.find_element(By.ID, "laTab").click()
time.sleep(2)
english_link = driver.find_element(By.XPATH, "//a[text()='English' and @class='mnuItm']")
driver.execute_script("arguments[0].click();", english_link)

[6755:4062986:0527/223405.503164:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


In [8]:
#### Change region ####
time.sleep(2)
driver.find_element(By.XPATH, "//a[text()='Region' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(2)

# 2. Type "United Kingdom" in the search box
region_input = driver.find_element(By.ID, "reTxt")
region_input.clear()
region_input.send_keys("United Kingdom")
time.sleep(2)  # wait for dropdown to appear

# 3. Click button
driver.find_element(By.ID, "reLkp").click()
time.sleep(4)

# 4 find uk in list and click it
driver.find_element(By.XPATH, "//a[text()='United Kingdom' and contains(@class, 'mnuItm')]").click()
time.sleep(2)

[6755:4062986:0527/223451.587855:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT


In [9]:
#### Industry ####
driver.find_element(By.XPATH, "//a[text()='Industry' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(2)

# 2. Type "Agriculture" in the input box
industry_input = driver.find_element(By.ID, "inTxt")
industry_input.clear()
industry_input.send_keys("Agriculture")
time.sleep(2)

# 3. Click the blue play/search icon
driver.find_element(By.ID, "inLkp").click()
time.sleep(2)

# 4. Click the "Agriculture" result
driver.find_element(By.XPATH, "//a[text()='Agriculture' and contains(@class, 'mnuItm')]").click()
time.sleep(2)

In [10]:
#### Subject ####

# Apply to all desired subjects
subjects_to_add = [
    "Commodity/Financial Market News",
    "Corporate/Industrial News",
    "Economic News"
]

def add_subject(subject_name):
    # Ensure the Subject tab is open before calling this
    subject_input = driver.find_element(By.ID, "nsTxt")
    subject_input.clear()
    subject_input.send_keys(subject_name)
    time.sleep(2)

    # Try clicking suggestion (strong) first
    try:
        driver.find_element(By.XPATH, f"//strong[text()='{subject_name}']/ancestor::td").click()
        print(f"Selected via suggestion: {subject_name}")
    except:
        # Fallback: click direct <a> tag
        driver.find_element(By.XPATH, f"//a[text()='{subject_name}' and contains(@class, 'mnuItm')]").click()
        print(f"Selected via <a>: {subject_name}")
    time.sleep(2)
    
    
# Step 1: Open the Subject tab once
driver.find_element(By.XPATH, "//a[text()='Subject' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(2)

for s in subjects_to_add:
    add_subject(s)
    
# Step 3 (optional): Close the Subject tab by clicking it again
driver.find_element(By.XPATH, "//a[text()='Subject' and contains(@class, 'fesTabLinkFix')]").click()
time.sleep(2)

Selected via suggestion: Commodity/Financial Market News
Selected via suggestion: Corporate/Industrial News
Selected via suggestion: Economic News


In [11]:
# Add key search terms in search box at the top
driver.execute_script("""
    let editor = ace.edit(document.getElementsByClassName('ace_editor')[0]);
    editor.setValue('rst=tukn');
""")


In [12]:
# Remove some of the "more options"

driver.find_element(By.XPATH, "//a[contains(text(), 'More Options')]").click()
time.sleep(2)

# Check the box if not already checked
checkbox = driver.find_element(By.ID, "isteo_bool")
if not checkbox.is_selected():
    checkbox.click()
    time.sleep(2)

In [13]:
# Click the search button at the bottom
time.sleep(2)
driver.find_element(By.XPATH, '//*[@id="btnSearchBottom"]').click()
time.sleep(7)

[6755:4062986:0527/223650.300848:ERROR:google_apis/gcm/engine/registration_request.cc:291] Registration response error message: DEPRECATED_ENDPOINT
